In [1]:
import numpy as np
import pandas as pd
import pypsa  
import os 
import sys  
import datetime
from pyomo.environ import Constraint

file_name =  os.path.basename(sys.argv[0])
directory = os.path.dirname(os.path.realpath(file_name))+'/Output'
n_file = 'Output/' + [s for s in os.listdir(directory) if 'EU_Network_created' in s][0]

n3 = pypsa.Network()
n3.import_from_netcdf(n_file)

n_storage = pd.read_csv('Output/n_storage.csv',index_col=0)
n_storage.index = pd.to_datetime(n_storage.index)

Indices = np.load('Output/indices.npy')
NTCs = pd.read_csv('Network_Data/NTCs.csv',index_col=0)

INFO:pypsa.io:Imported network EU_Network_created_2020-11-02.nc has buses, carriers, generators, lines, links, loads, storage_units


In [2]:
#initialize dictionary of buses per country
Country2Bus = {}
for i in n3.buses['country'].unique():
    Country2Bus[i] = n3.buses[n3.buses['country'] == i].index

#Get the list of storage units with significant standard deviation over the course of the year
Storage_units = n3.storage_units.loc[n_storage.std(axis=0)>100000].index

n3.storage_units['cyclic_state_of_charge'] = False

In [3]:
#Rules used for extra_functionality - gathered here because they are used at different times for different cases

def storage_rule(model,su):
    final = n_storage.loc[n3.snapshots[hour],su]
    start = n_storage.loc[n3.snapshots[hour-1],su]
    if (final-start > 0) and ('hydro' in su):
        inflow = n3.storage_units_t.inflow.loc[n3.snapshots[hour],su]
        if start + inflow > final:
            return model.state_of_charge[su,n3.snapshots[hour]] >= final
        else:
            return model.state_of_charge[su,n3.snapshots[hour]] >= start + inflow
    else:
        return model.state_of_charge[su,n3.snapshots[hour]] >= final
    
def NTC_rule(model, border):
    Limit = NTCs.iloc[hour][border]
    if Limit >= 0:
        c0 = border.split(' > ')[0]
        c1 = border.split(' > ')[1]
        pos_lines = n3.lines[(n3.lines['bus0'].isin(Country2Bus[c0])) & (n3.lines['bus1'].isin(Country2Bus[c1]))].index
        neg_lines = n3.lines[(n3.lines['bus0'].isin(Country2Bus[c1])) & (n3.lines['bus1'].isin(Country2Bus[c0]))].index
        pos_links = n3.links[(n3.links['bus0'].isin(Country2Bus[c0])) & (n3.links['bus1'].isin(Country2Bus[c1]))].index
        neg_links = n3.links[(n3.links['bus0'].isin(Country2Bus[c1])) & (n3.links['bus1'].isin(Country2Bus[c0]))].index   
        pos_lines_p = sum([model.passive_branch_p['Line',i,n3.snapshots[hour]] for i in pos_lines])
        neg_lines_p = sum([model.passive_branch_p['Line',i,n3.snapshots[hour]] for i in neg_lines])
        pos_links_p = sum([model.link_p[i,n3.snapshots[hour]] for i in pos_links])
        neg_links_p = sum([model.link_p[i,n3.snapshots[hour]] for i in neg_links])
        return pos_lines_p + pos_links_p - neg_lines_p - neg_links_p <= Limit
    else:
        return Constraint.Skip
    
def extra_functionality(network,snapshots):
    model = network.model
    model.storage = Constraint(Storage_units,rule=storage_rule)
    model.NTC = Constraint(NTCs.columns,rule=NTC_rule)

In [4]:
# Run optimization for desired timesteps
for i in Indices: 

    print('Hour: ',i)
    print(datetime.datetime.now())
    hour = i
    timesteps = np.array([hour])
    
    n3.storage_units['state_of_charge_initial'] = n_storage.iloc[hour-1]
    n3.lopf(n3.snapshots[timesteps],solver_name='gurobi',extra_functionality=extra_functionality)

print(datetime.datetime.now())

n3.export_to_netcdf('Output/n3_10hour.nc')
n3.generators_t.p.iloc[Indices].to_csv('Output/n3_gen_10h.csv')
n3.storage_units_t.p.iloc[Indices].to_csv('Output/n3_storage_10h.csv')

Hour:  2723
2020-11-11 14:48:37.408732


INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 790481266.5179131
  Upper bound: 790481266.5179131
  Number of objectives: 1
  Number of constraints: 14568
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 64064
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination me

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 1339471570.6507418
  Upper bound: 1339471570.6507418
  Number of objectives: 1
  Number of constraints: 14569
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 64066
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination 

...

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 8391659805.273265
  Upper bound: 8391659805.273265
  Number of objectives: 1
  Number of constraints: 14569
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 64066
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination me

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 20839245.696926296
  Upper bound: 20839245.696926296
  Number of objectives: 1
  Number of constraints: 14572
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 64078
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination 

INFO:pypsa.io:Exported network n3_10hour.nc has storage_units, carriers, links, loads, buses, lines, generators
